In [15]:
import cv2
import numpy as np

# Parámetros principales
T = 45  # Umbral de binarización
kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,5))
fps = 25  # Ajusta según tu vídeo

# Diccionario de líneas de carril
lines_dic = [
    {"cx1": 480, "cy1": 850, "cx2": 580, "cy2": 850},
    {"cx1": 610, "cy1": 850, "cx2": 720, "cy2": 850},
    {"cx1": 990, "cy1": 750, "cx2": 1090, "cy2": 750},
    {"cx1": 1320, "cy1": 900, "cx2": 1470, "cy2": 900},
    {"cx1": 1180, "cy1": 640, "cx2": 1210, "cy2": 640},
    {"cx1": 1420, "cy1": 730, "cx2": 1500, "cy2": 730},
    {"cx1": 1630, "cy1": 770, "cx2": 1770, "cy2": 770}
]

def line_crossed(centroidX, centroidY, line):
    return line["cx1"] <= centroidX <= line["cx2"] and abs(centroidY - line["cy1"]) <= 10

def speed_calc(prev_obj, actual_obj, fps):
    x_prev, y_prev, prev_frame = prev_obj
    x_actual, y_actual, actual_frame = actual_obj
    distance = ((x_actual - x_prev) ** 2 + (y_actual - y_prev) ** 2) ** 0.5
    time = (actual_frame - prev_frame) / fps
    return distance / time if time > 0 else 0

def find_car_id(cx, cy, car_tracks, max_distance=50):
    for car_id, positions in car_tracks.items():
        last_pos = positions[-1]
        x_last, y_last, _ = last_pos
        dist = ((cx - x_last)**2 + (cy - y_last)**2)**0.5
        if dist < max_distance:
            return car_id
    return None


In [16]:

video = cv2.VideoCapture('trafico01.mp4')
frames = []
while True:
    ret, frame = video.read()
    if not ret:
        break
    frames.append(frame)
video.release()

# Calcula fondo promedio
sumframe = None
count = 0
for frame in frames:
    framefloat = frame.astype(np.float32)
    if sumframe is None:
        sumframe = np.zeros_like(framefloat)
    sumframe += framefloat
    count += 1
fondo_promedio = sumframe / count
fondo_promedio_uint8 = fondo_promedio.astype(np.uint8)




In [5]:
car_counts = [0] * len(lines_dic)
car_tracks = {}  # id: [ (cx, cy, frame) ]
car_crossed_lines = {}  # id: set(line_idx)
car_speeds = {}  # id: [vels]
next_car_id = 0

In [18]:
for frame_idx, frame in enumerate(frames):
    # Binarización y limpieza morfológica
    diferencia = cv2.absdiff(frame, fondo_promedio_uint8)
    gris = cv2.cvtColor(diferencia, cv2.COLOR_BGR2GRAY)
    gris = cv2.GaussianBlur(gris, (5,5), 0)
    _, binario = cv2.threshold(gris, T, 255, cv2.THRESH_BINARY)
    binario = cv2.morphologyEx(binario, cv2.MORPH_OPEN, kernel)
    binario = cv2.morphologyEx(binario, cv2.MORPH_CLOSE, kernel)

    contornos, _ = cv2.findContours(binario, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img = frame.copy()

    # Dibuja líneas y contador
    for idx, line in enumerate(lines_dic):
        cv2.line(img, (line['cx1'], line['cy1']), (line['cx2'], line['cy2']), (60,180,255), 2)
        cv2.putText(img, f'Carril {idx+1}: {car_counts[idx]}', (line['cx1'], line['cy1']-20),
            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (180,180,180), 2)
    
    # Procesamiento de los objetos detectados
    for cnt in contornos:
        if cv2.contourArea(cnt) < 300:
            continue
        x, y, w, h = cv2.boundingRect(cnt)
        cx, cy = x + w//2, y + h//2

        # Tracking básico (empareja por proximidad, puedes mejorar con un tracker)
        car_id = next_car_id
        next_car_id += 1
        car_tracks[car_id] = [(cx, cy, frame_idx)]
        car_crossed_lines[car_id] = set()

        # Lógica de cruce y conteo de líneas
        for idx, line in enumerate(lines_dic):
            if idx not in car_crossed_lines[car_id] and line_crossed(cx, cy, line):
                car_counts[idx] += 1
                car_crossed_lines[car_id].add(idx)
                if len(car_tracks[car_id]) > 1:
                    v = speed_calc(car_tracks[car_id][-2], car_tracks[car_id][-1], fps)
                    if car_id not in car_speeds:
                        car_speeds[car_id] = []
                    car_speeds[car_id].append((idx, v))

        # Dibuja el bounding box y centroide
        cv2.rectangle(img, (x, y), (x+w, y+h), (0,255,255), 2)
        cv2.circle(img, (cx, cy), 4, (255,0,0), -1)
    
    img_resized = cv2.resize(img, (960, 540))  # Cambia (960, 540) por el tamaño que mejor se ajuste
    cv2.imshow('Detección y conteo', img_resized)

    if cv2.waitKey(25) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()
